## Familiarization with LLM packages such as `LangChain`, `OpenAI` & `tiktoken`

In [1]:
import credentials

### 1. Tiktoken

https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb


Encodings specify how text is converted into tokens. Different models use different encodings

`tiktoken` supports three encodings used by OpenAI models:

| Encoding name           | OpenAI models                                       |
|-------------------------|-----------------------------------------------------|
| `cl100k_base`           | `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002`  |
| `p50k_base`             | Codex models, `text-davinci-002`, `text-davinci-003`|
| `r50k_base` (or `gpt2`) | GPT-3 models like `davinci`                         |

In [26]:
import tiktoken

In [27]:
encoding = tiktoken.get_encoding("cl100k_base")
#encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [28]:
text = 'Tomorrow it will rain in Budapest.'
encoding.encode(text)

[91273, 433, 690, 11422, 304, 70695, 13]

In [29]:
text = 'TomorrowitwillraininBudapest.'
encoding.encode(text)

[91273, 275, 14724, 30193, 258, 33, 664, 28724, 13]

In [30]:
[encoding.decode_single_token_bytes(token) for token in [91273, 275, 14724, 30193, 258, 33, 664, 28724, 13]]

[b'Tomorrow', b'it', b'will', b'rain', b'in', b'B', b'ud', b'apest', b'.']

In [ ]:
[bytes.decode(encoding.decode_single_token_bytes(token)) for token in [91273, 275, 14724, 30193, 258, 33, 664, 28724, 13]]

['Tomorrow', 'it', 'will', 'rain', 'in', 'B', 'ud', 'apest', '.']

In [ ]:
encoding.decode([433])

' it'

In [ ]:
encoding.decode([275])

'it'

### 2. `LangChain`, `Pinecone` & other vector databases / vector representers for Document Representation

Load PDF

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
loader = PyPDFLoader("../docs/investment_report.pdf")
#doc = loader.load_and_split()
doc = loader.load_and_split(text_splitter = RecursiveCharacterTextSplitter(chunk_size = 4000, chunk_overlap = 200))

In [5]:
len(doc)

7

In [6]:
for i in doc:
    print(len(i.page_content))

2991
3986
549
3977
632
3869
2733


In [7]:
print(doc[0].page_content[:400])

Quarterly Investment Report 
April 2023
Despite banking instability and uncertain monetary policy expectations, the U.S. economy ended the quarter on  
a positive note. The S&P 500 Index rose 3.7% in March and 7.5% in the first quarter. The U.S. markets outperformed 
most major international markets. During the quarter, bond prices moved up, pushing Treasury yields in 1-year  and longer maturities


In [8]:
doc[0].metadata

{'source': '../docs/investment_report.pdf', 'page': 0}

In [9]:
page_num = doc[-1].metadata['page']
print('There are', page_num, 'pages in this PDF')

There are 3 pages in this PDF


Embeddings to use

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
openai_embedder = OpenAIEmbeddings(openai_api_key = credentials.openai_api)

In [ ]:
openai_embed_result = openai_embedder.embed_query('Embed this text!')
len(openai_embed_result)

In [ ]:
openai_embed_results = openai_embedder.embed_documents(['Embed this text!', 'This one too, please.'])
print(len(openai_embed_results))
print(len(openai_embed_results[0]))

In [12]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
hf_embedder = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

c:\Users\Rabay_Kristof\Documents\Python Scripts\openAI_experiments\openAI_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
hf_embed_result = hf_embedder.embed_query('Embed this text!')
len(hf_embed_result)

768

In [14]:
hf_embed_results = hf_embedder.embed_documents(['Embed this text!', 'This one too, please.'])
print(len(hf_embed_results))
print(len(hf_embed_results[0]))

2
768


Vector databases

https://python.langchain.com/en/latest/modules/indexes/vectorstores.html?highlight=vectorstores

For some, like `Pinecone`, we need an API connection.
For others we can simply store them locally, e.g. in a pickle

1. FAISS

In [15]:
from langchain.vectorstores import FAISS

In [16]:
db = FAISS.from_documents(doc, hf_embedder)

In [17]:
query = 'What happened in the bond market during the past quarter?'
relevant_docs = db.similarity_search(query)

print(len(relevant_docs))

4


In [18]:
for e, i in enumerate(db.similarity_search_with_score(query)):
    print(e, ' --- ', i[1])

0  ---  0.72051513
1  ---  0.87298733
2  ---  1.0739416
3  ---  1.0902194


In [19]:
for e, i in enumerate(db.similarity_search_with_relevance_scores(query)):
    print(e, ' --- ', i[1])

0  ---  0.4905188642503674
1  ---  0.38270473909543834
2  ---  0.24060862024283913
3  ---  0.22909848450190795


2. `Pinecone`

In [21]:
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(api_key = credentials.pinecone_api,
              environment = credentials.pinecone_loc)

In [ ]:
index_name = "first-experiment-sbert"
index = Pinecone.from_documents(doc, hf_embedder, index_name=index_name)

### 3. `OpenAI`

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name='text-davinci-003', openai_api_key=credentials.openai_api)

In [ ]:
#need billing set-up for API to be live
#llm('Tell me a joke')

In [ ]:
llm.get_num_tokens('Tell me a joke.')

5

In [ ]:
llm.get_num_tokens('Tellmeajoke.')

5